In [1]:
import os
import re
import csv
import sys
import csv
import random
import ast

csv.field_size_limit(sys.maxsize)

131072

In [2]:
# load data
recs = []
with open("../all-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
print len(recs)
recs[0]

39191


{'From': 'Kuwait',
 'action': "['establish']",
 'decision': 'support',
 'id': '0',
 'institutions': '[]',
 'notes': '',
 'text': '69.1 The establishment of an international round table to discuss the interrelation between security and fundamental freedoms (Kuwait). );',
 'theme': '[]',
 'to': 'Algeria',
 'year': '2008'}

## 1. Clean

In [3]:
# check for broken csv rows
for i in recs:
    if not i['year'].startswith('2'):
        print i['text']

In [33]:
# ordering
"""
numbs = []
for i in recs[:500]:
    n = i['text'].split()[0]
    n = n.split('.')
    i['n1'] = n[0]
    i['n2'] = n[1]
    if len(n) == 3:
        i['n3'] = n[2]
    else:
        i['n3'] = 0
"""

## 2. Sample

In [3]:
f = open('../sample-ids.txt','r')
sampled_ids = f.readlines()
f.close()
sampled_ids = [ast.literal_eval(i) for i in sampled_ids]
sampled_ids = [item for sublist in sampled_ids for item in sublist]

In [4]:
# remove already sampled recs
for i in recs:
    if i['id'] in sampled_ids:
        recs.remove(i)
len(recs)

38897

In [5]:
# sample
rec_ids = [int(i['id']) for i in recs] # get ids of remaining recs
ids = random.sample(rec_ids, 200) # sample those ids

sample = []
for i in recs:
    if int(i['id']) in ids:
        sample.append(i)
len(sample)

200

In [6]:
# delete data we don't want to include
for i in sample:
    del i['institutions']
    del i['theme']
    del i['action']
    del i['notes']
    
sample[0]

{'From': 'Albania',
 'decision': 'consider',
 'id': '296',
 'text': '60.1.1 Ratify the Optional Protocol to the Convention against Torture (United Kingdom, Albania, Mexico, Brazil);',
 'to': 'Japan',
 'year': '2008'}

In [7]:
#writing column headings
keys = sample[0].keys()

#writing the rest
with open('sample-matt-sentiment.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sample)

In [8]:
# add ids to the list of sampled data
ids = [str(i) for i in ids]
with open('../sample-ids.txt', 'a') as output_file:
    output_file.write("\n" + str(ids))

## 3. Test

In [88]:
# load data
recs = []
with open("../all-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
len(recs)
recs[0]

{'From': 'Kuwait',
 'action': "['establish']",
 'decision': 'support',
 'id': '0',
 'institutions': '[]',
 'notes': '',
 'text': '69.1 The establishment of an international round table to discuss the interrelation between security and fundamental freedoms (Kuwait). );',
 'theme': '[]',
 'to': 'Algeria',
 'year': '2008'}

In [89]:
# load Erin
erin = []
with open("erin-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                erin.append(row)
            
erin[1]

{'From': 'Algeria',
 'action': "['assistance']",
 'decision': 'consider',
 'id': '1430',
 'institutions': '[]',
 'notes': '',
 'sentiment': '2',
 'text': '94.37 Pursue its efforts, with the assistance and cooperation of the international community, for the realization of the rights of women and women\xeas empowerment as an important sector of society that can contribute significantly to the development of the country (Algeria). );',
 'theme': "['women']",
 'to': 'Bangladesh',
 'year': '2009'}

In [90]:
# making literal lists
for i in erin:
    try:
        i['theme'] = ast.literal_eval(i['theme'])
        i['institutions'] = ast.literal_eval(i['institutions'])
        i['action'] = ast.literal_eval(i['action'])
    except Exception,e:
        print i
erin[1]

{'From': 'Algeria',
 'action': ['assistance'],
 'decision': 'consider',
 'id': '1430',
 'institutions': [],
 'notes': '',
 'sentiment': '2',
 'text': '94.37 Pursue its efforts, with the assistance and cooperation of the international community, for the realization of the rights of women and women\xeas empowerment as an important sector of society that can contribute significantly to the development of the country (Algeria). );',
 'theme': ['women'],
 'to': 'Bangladesh',
 'year': '2009'}

In [91]:
# strip whitespace:
for i in erin:
    i['theme'] = [x.strip() for x in i['theme']]
    i['institutions'] = [x.strip() for x in i['institutions']]
    i['action'] = [x.strip() for x in i['action']]

In [92]:
# get ids
ids = [rec['id'] for rec in erin]

In [93]:
# get original sample
sample = []
for i in recs:
    if i['id'] in ids:
        sample.append(i)
sample[0]  

{'From': 'Algeria',
 'action': '[]',
 'decision': 'consider',
 'id': '371',
 'institutions': '[]',
 'notes': '',
 'text': '64.9 That the guarantee provided for the freedom of association and assembly be enshrined into law (Algeria);',
 'theme': "['civil-so']",
 'to': 'Korea',
 'year': '2008'}

In [94]:
for i in sample:
    i['theme'] = ast.literal_eval(i['theme'])
    i['institutions'] = ast.literal_eval(i['institutions'])
    i['action'] = ast.literal_eval(i['action'])

sample[1]

{'From': 'Algeria',
 'action': ['assistance'],
 'decision': 'consider',
 'id': '1430',
 'institutions': [],
 'notes': '',
 'text': '94.37 Pursue its efforts, with the assistance and cooperation of the international community, for the realization of the rights of women and women\xd5s empowerment as an important sector of society that can contribute significantly to the development of the country (Algeria). );',
 'theme': ['women'],
 'to': 'Bangladesh',
 'year': '2009'}

In [95]:
# zip
matching = []
for i in range(100):
    dic = erin[i].copy()
    
    # Theme
    dic['theme-erin'] = erin[i]['theme']
    dic['theme-rochelle'] = sample[i]['theme']
    del dic['theme']
    
    if set(dic['theme-erin']) == set(dic['theme-rochelle']):
        dic['theme-match'] = True
    elif set(dic['theme-erin']).intersection(dic['theme-rochelle']):
        dic['theme-match'] = "Partial"
    else:
        dic['theme-match'] = False
    
    # institutions
    dic['institution-erin'] = erin[i]['institutions']
    dic['institution-rochelle'] = sample[i]['institutions']
    del dic['institutions']
    
    if set(dic['institution-erin']) == set(dic['institution-rochelle']):
        dic['institution-match'] = True
    elif set(dic['institution-erin']).intersection(dic['institution-rochelle']):
        dic['institution-match'] = "Partial"
    else:
        dic['institution-match'] = False
    
    # actions
    dic['action-erin'] = erin[i]['action']
    dic['action-rochelle'] = sample[i]['action']
    del dic['action']
    
    if set(dic['action-erin']) == set(dic['action-rochelle']):
        dic['action-match'] = True
    elif set(dic['action-erin']).intersection(dic['action-rochelle']):
        dic['action-match'] = "Partial"
    else:
        dic['action-match'] = False
    
    matching.append(dic)

In [96]:
# make lists of matches, etc
theme_match = [i for i in matching if i['theme-match'] == True]
theme_partial = [i for i in matching if i['theme-match'] == "Partial"]
theme_false = [i for i in matching if i['theme-match'] == False]

institution_match = [i for i in matching if i['institution-match'] == True]
institution_partial = [i for i in matching if i['institution-match'] == "Partial"]
institution_false = [i for i in matching if i['institution-match'] == False]

action_match = [i for i in matching if i['action-match'] == True]
action_partial = [i for i in matching if i['action-match'] == "Partial"]
action_false = [i for i in matching if i['action-match'] == False]

In [97]:
print "Theme" + "\n"
print "Match: " + str(len(theme_match))
print "Partial: " + str(len(theme_partial))
print "False: " + str(len(theme_false))
print "\n"

print "Institutions" + "\n"
print "Match: " + str(len(institution_match))
print "Partial: " + str(len(institution_partial))
print "False: " + str(len(institution_false))
print "\n"

print "Action" + "\n"
print "Match: " + str(len(action_match))
print "Partial: " + str(len(action_partial))
print "False: " + str(len(action_false))
print "\n"

Theme

Match: 62
Partial: 28
False: 10


Institutions

Match: 81
Partial: 3
False: 16


Action

Match: 89
Partial: 5
False: 6




In [98]:
for i in theme_false:
    print i['id'] + ": " + i['text']
    print "ERIN: " + str(i['theme-erin'])
    print "ROCHELLE: " + str(i['theme-rochelle']) + "\n"

1814: 76.35 Share with all members of the Working Group its experience in increasing purchasing power and on efforts which have eliminated squatting in Cameroon (Sudan);
ERIN: ['infrastructure']
ROCHELLE: []

1827: 77.1 Further adopt additional measures, including granting suspensive effect to appeals against deportation orders, to ensure its full implementation without any exceptions of the principle of non-refoulement (Czech Republic);
ERIN: ['migrants']
ROCHELLE: []

3344: 77.1.3 accelerate the ratification of CPD (Azerbaijan);
ERIN: ['disabilities']
ROCHELLE: []

5762: 98.10. That legislation in Belarus regarding the registration of non.governmental organizations be streamlined and made less burdensome (Ireland);
ERIN: ['civil-so']
ROCHELLE: []

9702: 79.49. Continue its efforts in the area of providing welfare to older persons (Sudan);
ERIN: ['elderly', 'poverty']
ROCHELLE: []

15277: 98.25. Review its legislation on communications and bring it in line with international standards

## 4. Turn to Binary SpreadSheet

In [68]:
institutions = ['core', 'CRPD', 'ICCPR', 'eu-vaw', 'geneva', 'ICC', 'ILO', 'ICESR', 'achr', 'CAT', 'CRC', 'echr', '1954-stateless', 'traffick', 'CTOC', 'ICMW', 'ICERD', 'eu-child', 'sp-pr', 'PP', 'kampala', '1951refugees', 'pol-women', 'CPED', 'uncac', 'ottawa', '1961-stateless', 'CEDAW', 'unesco-ed', 'hague', 'genocide', 'HRC', 'unesco-expr', 'int-cord', 'frame-minorities', 'ind169', 'OP', 'ic-mercenaries', 'eu-migrant', 'un-indig', 'vienna', 'ouagadougou']

In [69]:
actions = ['accede', 'ratify', 'take-measures', 'sign', 'report', 'continue', 'withdraw', 'harmonize', 'party', 'implement', 'establish']

In [70]:
theme = ['death-pen', 'infrastructure ', 'intelligence', 'prisoners', 'elderly', 'culture', 'education', 'children', 'civil-so', 'disappearances', 'police', 'hiv', 'privacy', 'discrimination', 'impunity', 'governance', 'labor', 'environment', 'religion', 'foreign-movement', 'health', 'speech', 'domest-movement', 'terrorism', 'paramilitary', 'corruption', 'sex-violence', 'civil-conflict', 'race', 'torture', 'food', 'extra-judicial', 'judiciary', 'indigenous', 'lgbt', 'self-determ', 'poverty', 'ethnic', 'women', 'land', 'reproductive', 'post-conflict', 'disabilities', 'migrants', 'pol-imprisonment', 'socialism', 'trafficking', 'military', 'property']

In [71]:
erin[0].keys()

['From',
 'sentiment',
 'text',
 'decision',
 'id',
 'to',
 'theme',
 'notes',
 'year',
 'action',
 'institutions']

In [72]:
for i in sample:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0 
    
    del i['to']
    del i['From']
    del i['text']
    del i['notes']
    del i['action']
    del i['theme']
    del i['decision']
    del i['institutions']
    del i['year']

In [73]:
for i in erin:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0  
            
    del i['to']
    del i['From']
    del i['text']
    del i['year']
    del i['institutions']
    del i['action']
    del i['theme']
    del i['decision']
    del i['sentiment']

In [74]:
## writing sample

#writing column headings
keys = sample[0].keys()

#writing the rest
with open('sample-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sample)

In [75]:
## writing erin

#writing column headings
keys = erin[0].keys()

#writing the rest
with open('erin-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(erin)